# Viagens de pedestres versus outros meios de transporte

Caderno Jupyter referente à **Issue #5**: *Mapa coroplético de origens e destinos de pedestres com filtros e granularidades* 

**APENAS MUNICÍPIO DE SÃO PAULO**

Comparação de viagens de pedestres vs viagens por outros meios de transporte por região (zona OD ou distrito)

---

In [ ]:
import pandas as pd
import geopandas
import folium

### Carregando datasets e convertendo para CRS em comum 

Carregando datasets

In [ ]:
od23_path = "../../data/OD/ODS/od23_all.csv" 
od = pd.read_csv(od23_path)

od_districts_shapefile_path = "../../data/OD/OD-2023/002_Site Metro Mapas_190225/Shape/Distritos_2023_region.shp" 
od_districts = geopandas.read_file(od_districts_shapefile_path)

od_zones_shapefile_path = "../../data/OD/OD-2023/002_Site Metro Mapas_190225/Shape/Zonas_2023.shp" 
od_zones = geopandas.read_file(od_zones_shapefile_path)

Filtrando zonas OD que pertencem ao município de SP

In [ ]:
od_zones = od_zones.query('NumeroMuni == 36')

Convertendo CRS

In [ ]:
od_districts = od_districts.to_crs("EPSG:4326")
od_zones = od_zones.to_crs("EPSG:4326")

### Cálculo de total de viagens por zonas OD

In [ ]:
fe_via_all_sum_zone = (
    od.groupby("ZONA_O")["FE_VIA"].sum()
    .reset_index()
    .rename(columns={"ZONA_O": "NumeroZona", "FE_VIA": "TOTAL_VIAGENS"})
)

Calcular viagens de pedestres vs outros meios de transportes

- Por zona (ZONA_O)
- Por distrito

### Viagens por zona OD

In [ ]:
# 1) Filtrando apenas pedestres 
pedestrians_walk = od.query('MODOPRIN == 17')

# 2) Total de viagens para pedestres
zone_walk = (
    pedestrians_walk.groupby("ZONA_O")["FE_VIA"].sum()
    .reset_index()
    .rename(columns={"ZONA_O": "NumeroZona", "FE_VIA": "VIAGENS_PED"})
)

# 3) Merge do total de viagens / total de viagens de pedestres por zonas OD
zone_share = zone_walk.merge(
    fe_via_all_sum_zone,
    on="NumeroZona",
    how="inner"
)

# 4) Cálculo da representatividade de pedestres
zone_share["PERC_PED"] = 100 * zone_share["VIAGENS_PED"] / zone_share["TOTAL_VIAGENS"]

# 5) Cálculo da representatividade dos outros meios de transporte
zone_share["PERC_OUTROS"] = 100 - zone_share["PERC_PED"] 

# 6) Ordena decrescentemente por percentual de viagens de pedestres
zone_share = zone_share.sort_values("PERC_PED", ascending=False)
zone_share.head()

### Viagens por distrito

In [ ]:
# 1) Junta zonas com distritos para agregação futura de distritos
zone_district = od_zones[["NumeroZona", "NumDistrit", "NomeDistri"]].merge(
    zone_share, on="NumeroZona", how="left"
).fillna({"VIAGENS_PED": 0, "PERC_PED": 0, "PERC_OUTROS": 0, "TOTAL_VIAGENS": 0})

# 2) Agregação de viagens de pedestres por distrito
district_walk = (
    zone_district.groupby(["NumDistrit", "NomeDistri"])["VIAGENS_PED"].sum()
    .reset_index()
)

# 3) Total de viagens de todos os meios por distrito
fe_via_all_sum_district = (
    zone_district.groupby("NumDistrit")["TOTAL_VIAGENS"].sum()
    .reset_index()
)

# 4) Merge do total de viagens / total de viagens de pedestres por distritos
district_share = district_walk.merge(
    fe_via_all_sum_district,
    on="NumDistrit",
    how="inner"
)

# 5) Cálculo da representatividade de pedestres
district_share["PERC_PED"] = 100 * district_share["VIAGENS_PED"] / district_share["TOTAL_VIAGENS"]

# 6) ordena decrescentente de acordo com o percentual
district_share = district_share.sort_values("PERC_PED", ascending=False)
district_share.head()

## Merge com geometrias

Juntar com geometrias

In [ ]:
# zonas
zones_map = od_zones.merge(zone_share, on="NumeroZona", how="left").fillna(0)

# distritos
districts_map = od_districts.merge(district_share, left_on="NumeroDist", right_on="NumDistrit", how="left").fillna(0)

Filtrando distritos que pertençam São Paulo

In [ ]:
districts_map = districts_map.query('PERC_PED > 0')

## Mapa

Criar mapa Folium com LayerControl

In [ ]:
# centro aproximado (SP)
m = folium.Map(location=[-23.55, -46.63], zoom_start=10)

Camada Zona OD

In [ ]:
# camada por zona
folium.Choropleth(
    geo_data=zones_map.to_json(),
    name="Zonas OD",
    data=zones_map,
    columns=["NumeroZona", "PERC_PED"],
    key_on="feature.properties.NumeroZona",
    fill_color="YlGnBu",
    fill_opacity=0.7,
    line_opacity=0.3,
    legend_name="Viagens de pedestres (%) - Zona OD"
).add_to(m)

# adiciona tooltip + popup para zonas
folium.GeoJson(
    zones_map,
    style_function=lambda x: {"fillOpacity": 0, "color": "transparent"}, 
    tooltip=folium.GeoJsonTooltip(
        fields=["NomeZona", "PERC_PED"],
        aliases=["Zona OD:", "Representatividade (%)"],
        localize=True,
        sticky=True
    ),
    popup=folium.GeoJsonPopup(
        fields=["NomeZona", "VIAGENS_PED", "PERC_PED"],
        aliases=["Zona OD:", "Viagens:", "Representatividade (%)"],
        localize=True
    ),
    name="zonas od - representatividade",
    show=True
).add_to(m)

Camada Distrito

In [ ]:
# camada por distrito
folium.Choropleth(
    geo_data=districts_map.to_json(),
    name="Distritos",
    data=districts_map,
    columns=["NumeroDist", "PERC_PED"],
    key_on="feature.properties.NumeroDist",
    fill_color="OrRd",
    fill_opacity=0.7,
    line_opacity=0.3,
    legend_name="Viagens de pedestres (%) - Distritos",
    show=False
).add_to(m)

# adiciona tooltip + popup para distritos
folium.GeoJson(
    districts_map,
    style_function=lambda x: {"fillOpacity": 0, "color": "transparent"},
    tooltip=folium.GeoJsonTooltip(
        fields=["NomeDistri_x", "PERC_PED"],
        aliases=["Distrito:", "Representatividade (%)"],
        localize=True,
        sticky=True
    ),
    popup=folium.GeoJsonPopup(
        fields=["NomeDistri_x", "VIAGENS_PED", "PERC_PED"],
        aliases=["Distrito:", "Viagens pedestres:", "Representatividade (%)"],
        localize=True
    ),
    name="distrito - representatividade",
    show=False
).add_to(m)

Salvar mapa

In [ ]:
# adicionar controles
folium.LayerControl().add_to(m)

m.save("../built_maps/choropleth_ped_trips_maps/ped_vs_others_trips.html")